<a href="https://colab.research.google.com/github/VorobyvEgor/Seminar_Sber/blob/main/Seminars/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Данные
Для демонстраций загрузим набор данных [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/Automobile). В данных присутствуют категориальные, целочисленные и вещественнозначные признаки.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

--2022-02-17 17:31:26--  https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25936 (25K) [application/x-httpd-php]
Saving to: ‘imports-85.data’

imports-85.data     100%[===================>]  25.33K  --.-KB/s    in 0.07s   

2022-02-17 17:31:26 (375 KB/s) - ‘imports-85.data’ saved [25936/25936]



In [ ]:
! ls

imports-85.data  sample_data


In [ ]:
data = pd.read_csv("imports-85.data", \
                    header=None,  na_values=["?"])
print(data.shape)
data.head()

(205, 26)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [ ]:
data[data[25].isna()]

In [ ]:
data = data[data[25].notna()]
print(data.shape)

In [ ]:
y = data[25] # цена
print(y.shape)
X = data.drop(25, axis=1)
print(X.shape)

## Предобработка данных
Предобработка данных важна при применении любых методов машинного обучения, а в особенности для линейных моделей. В sklearn предобработку удобно делать с помощью модуля [preprocessing](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) или методов библиотеки pandas.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

In [ ]:
X.dtypes

In [ ]:
cat_features = list(X.dtypes[X.dtypes == "object"].index)
cat_features

In [ ]:
real_features = list(set(X.columns) - set(cat_features))
real_features

In [ ]:
# Для категориальных - пустыми строками
X.loc[:, cat_features] = X.loc[:, cat_features].fillna("")
X.head()

#### Кодирование категориальных переменных
В Sklearn имплементирвоано два основных метода:
* One-hot-encoding - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
* Label_encoding - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

В pandas: pd.get_dummies()


In [ ]:
X_dum = pd.get_dummies(X, drop_first=True)
print(X_dum.shape)
X_dum.head()

In [ ]:
# Делим на train \ test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)

### Заполнение пропусков
В матрице объекты-признаки могут быть пропущенные значения, и это вызовет исключение при попытке передать такую матрицу в функцию обучения модели или даже предобработки. Если пропусков немного, можно удалить объекты с пропусками из обучающей выборки. Заполнить пропуски можно разными способами:
* заполнить средними (mean, median);
* предсказывать пропущенные значения по непропущенным.

Последний вариант сложный и применяется редко. Для заполнения константами можно использовать метод датафрейма fillna, для замены средними - класс preprocessing.Imputer.

Заполнение на train выборке

In [ ]:
# Для вещественнозначных признаков заполним пропуски средними
X_train_real = X_train[real_features]
mis_replacer = SimpleImputer(strategy="mean")
mis_replacer.fit(X_train_real)

X_train_no_mis = X_train.copy()
X_train_no_mis.loc[:, real_features] = mis_replacer.transform(X_train_real)

In [ ]:
print(X_train_no_mis.shape)
X_train_no_mis.head()

Заполнение на test выборке

In [ ]:
# Для вещественнозначных признаков заполним пропуски средними
X_test_real = X_test[real_features]

X_test_no_mis = X_test.copy()
X_test_no_mis.loc[:, real_features] = mis_replacer.transform(X_test_real)

print(X_test_no_mis.shape)
X_test_no_mis.head()

Всегда нужно осознавать, случайны ли пропуски в каком-то признаке. Иногда факт отсутствия информации о значении признака может сам быть важным признаком, который необходимо добавить к другим признакам.

__Пример:__ предсказание возраста пользователя по данным с его телефона. Поскольку люди старшего возраста чаще пользуются простыми телефонами, факт отсутствия каких-то данных (например, истории посещенных интернет-страниц), скорее всего, будет хорошим признаком.

Для категориальных признаков рекомендуется создавать отдельную категорию, соответствующую пропущенному значению. В наши данных пропусков в категориальных признаках нет.

In [ ]:
print(X_train_no_mis.shape)
print(X_test_no_mis.shape)

(160, 65)
(41, 65)


### Масштабирование признаков
При начале работы с данными всегда рекомендуется приводить все признаки к одному масштабу.  Это важно по нескольким причинам:
* ускорение обучения модели
* улучшение численной устойчивости при работе с матрицей объекты-признаки (рядом с нулем чисел с плавающей точкой больше, чем с области больших чисел)
* для линейных моделей: интерпретация весов при признаках как меры их значимости

Первый популярный способ масштабирования - нормализация: вычитание среднего из каждого признака и деление на стандартное отклонение (StandardScaler в sklearn). Второй популярный способ: вычитание минимума из каждого признака, а затем деление на разницу максимального и минимального значения (MinMaxScaler в sklearn).

In [ ]:
normalizer = preprocessing.MinMaxScaler()
normalizer.fit(X_train_no_mis)

X_real_norm_np = normalizer.transform(X_train_no_mis)
X_train_transformed = pd.DataFrame(data=X_real_norm_np)

X_real_norm_np = normalizer.transform(X_test_no_mis)
X_test_transformed = pd.DataFrame(data=X_real_norm_np)

In [ ]:
print(X_train_transformed.shape)
X_train_transformed.head()

In [ ]:
print(X_test_transformed.shape)
X_test_transformed.head()

### Добавление признаков
Особенно важным моментом для линейной регрессии является нелинейное преобразование признаков. Это позволяет использовать линейную регрессию для моделирования нелинейных зависимостей. Наиболее популярны следующие преобразования: полиномиальные признаки (PolynomialFeatures в sklearn), взятие логарифма, квадратного корня, применение тригонометрических функий.

Например, в нашем датасете зависимость целевой перемнной от шестого признака скорее квадратичная, чем линейная:

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(4):
  for j in range(4):
    plt.subplot(4, 4, i+4*j+1)
    plt.scatter(X_train_transformed[i+4*j], y_train);
# 3, 6, 11 - скорее квадратичные
# 13, 14 - 1/x или  1/sqrt(x)

In [ ]:
i = 6
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.scatter(X_train_transformed[i], y_train)
plt.subplot(1, 2, 2)
plt.scatter(X_train_transformed[i]**2, y_train);

А для признака номер 13 линеаризовать зависимость получается с помощью функции $\frac 1 {\sqrt{\cdot}}$

In [ ]:
plt.figure(figsize=(18, 4))
plt.subplot(1, 3, 1)
plt.scatter(X_train_transformed[13], y_train)
plt.subplot(1, 3, 2)
plt.scatter(1/np.sqrt(X_train_transformed[13]), y_train)
plt.subplot(1, 3, 3)
plt.scatter(1/X_train_transformed[13], y_train);

In [ ]:
plt.figure(figsize=(18, 4))
plt.subplot(1, 3, 1)
plt.scatter(X_train_transformed[14], y_train)
plt.subplot(1, 3, 2)
plt.scatter(1/np.sqrt(X_train_transformed[14]), y_train)
plt.subplot(1, 3, 3)
plt.scatter(1/X_train_transformed[14], y_train);

Обратите внимание, что при генерации полиномиальных признаков матрица объекты-признаки может занимать очень много памяти.

### Обучаем модель и смотрим качество

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
lr = LinearRegression()
lr.fit(X_train_transformed, y_train)

predict_train = lr.predict(X_train_transformed)
predict_test = lr.predict(X_test_transformed)

print(np.sqrt(mean_squared_error(y_train, predict_train)))
print(np.sqrt(mean_squared_error(y_test, predict_test)))

In [ ]:
print(mean_absolute_error(y_train, predict_train))
print(mean_absolute_error(y_test, predict_test))

In [ ]:
lr.coef_

In [ ]:
X_train_transformed.corr().style.background_gradient(cmap='coolwarm')

Негативные последствия мультиколлинеарности для линейной регрессии:
* Увеличивается разброс коэффициентов, появляются большие положительные и большие отрицательные коэффициенты.
По абсолютной величине коэффициента становится невозможно судить о степени важности признака fj
* Коэффициенты утрачивают интерпретируемость.
* Повышается неустойчивость решения. Малые изменения данных, такие как добавление нового обучающего объекта или шумовые искажения значений признаков или ответов, способны существенно изменить вектор коэффициентов.
* В результате понижается обобщающая способность алгоритма.

Три основных подхода к устранению мультиколлинеарност:
* Регуляризация. Накладываются дополнительные ограничения на норму вектора коэффициентов. Это приводит к гребневой регрессии или методу
лассо. В первом случае задействуются все признаки, но снижается
эффективная размерность пространства. Во втором случае у части признаков
обнуляются коэффициенты, что равносильно их исключению из модели.
  - Гребневая регрессия: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
  - Лассо регрессия: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
* Преобразование признаков. Исходные n признаков с помощью некоторых преобразований переводятся в меньшее число m новых признаков. В частности,
линейные преобразования приводят к методу главных компонент.
* Отбор признаков. Производится явный перебор всевозможных подмножеств
признаков.